# Assignment 1 Workbook
   

Load data from the cloud

In [ ]:
# !apt-get install openjdk-8-jdk-headless -qq > /dev/null
# !wget -q http://mirror.cc.columbia.edu/pub/software/apache/spark/spark-3.0.0-preview2/spark-3.0.0-preview2-bin-hadoop2.7.tgz
# !tar xf spark-3.0.0-preview2-bin-hadoop2.7.tgz
# !pip install -q findspark
# !rm -rf spark-3.0.0-preview2-bin-hadoop2.7.tgz

In [34]:
import os
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.0.0-preview2-bin-hadoop2.7"

#import findspark
# findspark.init("spark-3.0.0-preview2-bin-hadoop2.7")# SPARK_HOME

from pyspark.sql import SparkSession
from pyspark import SparkContext

spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = SparkContext.getOrCreate()

In [ ]:
ls -lah

In [ ]:
!ls -la

In [ ]:
!install conda install -c anaconda wget
!wget https://s3.amazonaws.com/metcs777/taxi-data-sorted-small.csv.bz2

In [ ]:
#check if spark are running

print(sc.version)


In [35]:
lines = sc.textFile("Assignment1_dt/taxi-data-sorted-small.csv.bz2")
taxilines = lines.map(lambda line: line.split(","))

In [ ]:
#print first row of the data to make sure it is read in correctly
print(taxilines.take(2))

Use pycode from assignment 1 document to clean up the data loaded

In [36]:
#clean up 
#handling wrong data lines
def isfloat(val):
    try: 
        float(val)
        return True
    except:
        return False
    
#use isFloat to remove lines if they dont have 16 values and a float value !=0 for col 6 and col 12
def correctRows(p):
    if(len(p)==17):
        if(isfloat(p[5]) and isfloat(p[11])):
            if(float(p[5])!=0 and float(p[11])!=0):
                return p
            
#go thru each element per row and keep only the correctRows
cleanTaxiDT = taxilines.filter(correctRows)

### Part1: Top-10 Active Taxis
Many different taxis have had multiple drivers. Write and execute a Spark Python program that computes
the top ten taxis that have had the largest number of drivers. Your output should be a set of (medallion,
number of drivers) pairs


In [37]:
#set up each row with an 1 index
rdd_1 = cleanTaxiDT.map(lambda x: ((x[0], x[1]), 1))

In [38]:
rdd_1.first()

(('07290D3599E7A0D62097A346EFCC1FB5', 'E7750A37CAB07D0DFF0AF7E3573AC141'), 1)

In [39]:
#reduce, aggrgate by the combination of taxi and taxi drivers into a list
rdd_1_reduce = rdd_1.reduceByKey(lambda x, y: x+y)


In [40]:
# reduce in RDD to count taxi and most frequently used taxi
by_medalionRDD2 = rdd_1_reduce.map(lambda x: (x[0][0],1)).reduceByKey(lambda x, y: x+y)

In [41]:
#return the top 10 most used taxi by the count
top10taxi = sc.parallelize(by_medalionRDD2.top(10, lambda x: x[1]))

top10taxi.coalesce(1).saveAsTextFile("top10Taxi/")

### Part2: Top-10 Best Drivers
We would like to figure out who the top 10 best drivers are in terms of their average earned money per minute spent carrying a customer. The total amount field is the total money earned on a trip. In the end, we
are interested in computing a set of (driver, money per minute) pairs

Create a second clean RDD to remove any rows that contain no value for the minute column and save to cleanTaxiDT2

In [42]:
def correctRows2(p):
  if(float(p[4]) !=0):
    return p
cleanTaxiDT2 = cleanTaxiDT.filter(correctRows2)

In [ ]:
# check to see if the cleaned up data contail any zero value for column 5
cleanTaxiDT2.filter(lambda x: float(x[4])==0 ).count()

In [43]:
#get a driver rdd and map a function to get the money per minute value
by_drvr = cleanTaxiDT2.map(lambda x: (x[1], (float(x[11])/(float(x[4])/60), 1.0)))
#aggregate by key=driverID and tke an average using the aggregate byByKey
aggfun = lambda x, y: (x[0]+y[0], x[1]+y[1])
rateRDD = by_drvr.aggregateByKey((0.0,0.0), aggfun, aggfun)

avg_rateRDD = rateRDD.map(lambda x: (x[0], x[1][0]/x[1][1]))


In [44]:
# Get top 10 drivers
top10drivers = sc.parallelize(avg_rateRDD.top(10, lambda x: x[1]))
top10drivers.coalesce(1).saveAsTextFile("top10Drivers/")

### Part3: Best Day to Work
We would like to know which hour of the day is the best time for drivers that has the highest profit per miles. Consider the surcharge amount in dollar for each taxi ride (without tip amount) and the distance in miles, and sum up the rides for each hour of the day (24 hours) – consider the pickup time for your calculation. The profit ratio is the ration surcharge in dollar divided by the travel distance in miles for each specific time
of the day. 

Profit Ratio = (Surcharge Amount in US Dollar) / (Travel Distance in miles)
We are interested to know the time of the day that has the highest profit ratio.

In [45]:
#col index: surcharge=12, Distance=5, pickup datetime=2

# get the day out of the datetime column
from datetime import datetime

In [46]:

# create a method to extract hour
def getHour(x):
  dtObj = datetime.strptime(x, '%Y-%m-%d %H:%M:%S')
  return dtObj.strftime('%H')

#create aRDD with the relevant columns: pick up date, surcharge, distance
pt3RDD = cleanTaxiDT.map(lambda x: (getHour(x[2]), (float(x[12]), float(x[5]))))
#RDD to contain sum of all surcharge, all distance for each date as key
byHourRDD = pt3RDD.reduceByKey(lambda x, y: (x[0]+y[0], x[1]+y[1]))
#RDD to produce date and profit ratio by day
bestDayRDD = byHourRDD.map(lambda x: (x[0], x[1][0]/x[1][1]))



In [47]:
bestWrkHr = sc.parallelize(bestDayRDD.top(10, lambda x: x[1]))

bestWrkHr.coalesce(1).saveAsTextFile("top10WorkHours/")

In [ ]:
del byDateRDD


### Part4: Bonus 
How many percent of taxi customers pay with cash and how many percent using electronic cards?
Analyze these payment methods for different time of the day and provide a list of percents for each
day time? 

As a result provide two numbers for total percentages and a list like (hour of day, percent paid card) We would like to measure the efficiency of taxis drivers by finding out their average earned money per mile. (Consider the total amount which includes tips, as their earned money) Implement a Spark job
that can find out the top-10 efficient taxi divers.

What are mean, median, first and third quantiles of tip amount? How do find the median?
Using the IQR outlier detection method find out the top-10 outliers.

In [ ]:
# cash versus credit
# columns: pickup date = 2, payment type = 10

